<a href="https://colab.research.google.com/github/DOJO-Smart-Ways/DOJO-Beam-Transforms/blob/main/unit_test_pipeline_components/RenameColumns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install apache_beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 20.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.2/677.2 kB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 81.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup

In [29]:
import unittest
from apache_beam.testing.test_pipeline import TestPipeline
from apache_beam.testing.util import assert_that
from apache_beam.testing.util import equal_to
import apache_beam as beam
from datetime import datetime, timedelta
import pandas as pd

class RenameColumns(beam.DoFn):
    def __init__(self, column_mapping):
        self.column_mapping = column_mapping

    def process(self, element):
        new_element = {self.column_mapping.get(k, k): v for k, v in element.items()}
        yield new_element

class TestRenameColumns(unittest.TestCase):
    def test_rename_columns_full_mapping(self):
        with TestPipeline() as p:
            input_data = [{'a': 1, 'b': 2}]
            expected_data = [{'x': 1, 'y': 2}]
            column_mapping = {'a': 'x', 'b': 'y'}

            output = (
                p
                | "CreateInput" >> beam.Create(input_data)
                | "RenameColumns" >> beam.ParDo(RenameColumns(column_mapping))
            )

            assert_that(output, equal_to(expected_data))

    def test_rename_columns_partial_mapping(self):
        with TestPipeline() as p:
            input_data = [{'a': 1, 'b': 2}]
            expected_data = [{'x': 1, 'b': 2}]
            column_mapping = {'a': 'x'}

            output = (
                p
                | "CreateInput" >> beam.Create(input_data)
                | "RenameColumns" >> beam.ParDo(RenameColumns(column_mapping))
            )

            assert_that(output, equal_to(expected_data))

    def test_rename_columns_no_mapping(self):
        with TestPipeline() as p:
            input_data = [{'a': 1, 'b': 2}]
            expected_data = [{'a': 1, 'b': 2}]
            column_mapping = {}

            output = (
                p
                | "CreateInput" >> beam.Create(input_data)
                | "RenameColumns" >> beam.ParDo(RenameColumns(column_mapping))
            )

            assert_that(output, equal_to(expected_data))


    def test_rename_columns_dtype_consistency_with_timestamp(self):
        with TestPipeline() as p:
            initial_time_datetime = datetime.now()
            timestamp = pd.to_datetime(datetime.now())

            input_data = [{'datetime': initial_time_datetime, 'timestamp': timestamp}]
            expected_data = [{'new_datetime': initial_time_datetime, 'new_timestamp': timestamp}]
            column_mapping = {'datetime': 'new_datetime', 'timestamp': 'new_timestamp'}

            output = (
                p
                | beam.Create(input_data)
                | beam.ParDo(RenameColumns(column_mapping))
            )

            assert_that(output, equal_to(expected_data))




def run_tests():
    suite = unittest.TestSuite()
    loader = unittest.TestLoader()
    suite.addTests(loader.loadTestsFromTestCase(TestRenameColumns))

    runner = unittest.TextTestRunner()
    runner.run(suite)

# Chamada da função de execução dos testes
run_tests()

....
----------------------------------------------------------------------
Ran 4 tests in 2.399s

OK
